In [2]:
from ipyleaflet import Map, DrawControl, basemaps, basemap_to_tiles, LayersControl, TileLayer
from ipywidgets import Dropdown, VBox, HBox, Output
import json

# 1. Dropdowns iniciales
idioma = Dropdown(
    options=[('Español', 'es'), ('English', 'en')],
    description='Idioma:',
    layout={'width': '200px'}
)

tipo = Dropdown(
    description='Uso:',
    layout={'width': '350px'}
)

# 2. Diccionario de categorías por idioma
categorias = {
    'es': ['Casa', 'Parque', 'Fraccionamiento', 'Rascacielos', 'Edificio histórico',
           'Jardín', 'Tienda', 'Supermercado', 'Canal', 'Río', 'Palacio', 'Ayuntamiento'],
    'en': ['House', 'Park', 'Residential Complex', 'Skyscraper', 'Historical Building',
           'Garden', 'Store', 'Supermarket', 'Canal', 'River', 'Palace', 'City Hall']
}

# 3. Función para actualizar categorías según idioma
def actualizar_categorias(change):
    tipo.options = categorias[idioma.value]
    tipo.value = tipo.options[0]

idioma.observe(actualizar_categorias, names='value')
actualizar_categorias(None)

# 4. Capas base incluyendo personalizadas
capas_base = {
    "OpenStreetMap": basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    "Satélite (Esri)": basemap_to_tiles(basemaps.Esri.WorldImagery),
    "Topográfico (Esri)": basemap_to_tiles(basemaps.Esri.WorldTopoMap),
    "Carto Positivo": basemap_to_tiles(basemaps.CartoDB.Positron),
    "Carto Oscuro": basemap_to_tiles(basemaps.CartoDB.DarkMatter),
    "Topográfico (OSM)": TileLayer(
        url="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
        attribution="© OpenTopoMap contributors"
    ),
    "Transporte": TileLayer(
        url="https://tile.memomaps.de/tilegen/{z}/{x}/{y}.png",
        attribution="© MemoMap & Public Transport Tiles"
    )
}

# 5. Dropdown para selector de capas base
selector_capa = Dropdown(
    options=list(capas_base.keys()),
    value="OpenStreetMap",
    description="Mapa:"
)

# 6. Crear el mapa
m = Map(center=(0, 0), zoom=2, min_zoom=1, max_zoom=22)
capa_base_actual = capas_base[selector_capa.value]
m.add_layer(capa_base_actual)

# 7. Actualizar capa base cuando cambia
def actualizar_capa(change):
    global capa_base_actual
    nueva_capa = capas_base[change.new]
    m.substitute_layer(capa_base_actual, nueva_capa)
    capa_base_actual = nueva_capa

selector_capa.observe(actualizar_capa, names='value')
m.add_control(LayersControl(position='topright'))

# 8. Control de dibujo
draw_control = DrawControl()
m.add_control(draw_control)

# 9. Output y guardado
salida = Output()

@draw_control.on_draw
def guardar_clasificado(target, action, geo_json):
    geo_json['properties'] = {
        'uso': tipo.value,
        'idioma': idioma.value
    }
    with open("clasificado.geojson", "a") as f:
        json.dump(geo_json, f)
        f.write("\n")
    with salida:
        print(f"✅ Guardado: {tipo.value} ({idioma.label})")

# 10. Mostrar interfaz
VBox([
    HBox([idioma, tipo, selector_capa]),
    m,
    salida
])

In [ ]:
%pip install ipyleaflet

Note: you may need to restart the kernel to use updated packages.
